In [4]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import ipywidgets as widgets
from IPython.display import display

# 1. Definir las variables de entrada y salida
velocidad = ctrl.Antecedent(np.arange(0, 101, 1), 'velocidad')  # km/h
rpm = ctrl.Antecedent(np.arange(0, 7001, 1), 'rpm')  # rpm
pendiente = ctrl.Antecedent(np.arange(0, 101, 1), 'pendiente')  # % de inclinación (100% = 45°)
carga = ctrl.Antecedent(np.arange(0, 101, 1), 'carga')

# La salida sera el valor crisp de la marcha
marcha = ctrl.Consequent(np.arange(0, 101, 1), 'marcha')

# 2. Definir los conjuntos difusos
velocidad['baja'] = fuzz.trimf(velocidad.universe, [0, 0, 40])
velocidad['media'] = fuzz.trimf(velocidad.universe, [20, 50, 80])
velocidad['alta'] = fuzz.trimf(velocidad.universe, [60, 100, 100])  

rpm['bajas'] = fuzz.trimf(rpm.universe, [0, 0, 3000])
rpm['medias'] = fuzz.trimf(rpm.universe, [2000, 4000, 6000])
rpm['altas'] = fuzz.trimf(rpm.universe, [5000, 7000, 7000])

pendiente['llano'] = fuzz.trimf(pendiente.universe, [0, 0, 40])
pendiente['moderado'] = fuzz.trimf(pendiente.universe, [20, 50, 80])
pendiente['empinado'] = fuzz.trimf(pendiente.universe, [60, 100, 100])

carga['ligera'] = fuzz.trimf(carga.universe, [0, 0, 40])
carga['media'] = fuzz.trimf(carga.universe, [20, 50, 80])
carga['pesada'] = fuzz.trimf(carga.universe, [60, 100, 100])

marcha['baja'] = fuzz.trimf(marcha.universe, [0, 0, 40])
marcha['media'] = fuzz.trimf(marcha.universe, [20, 50, 80])
marcha['alta'] = fuzz.trimf(marcha.universe, [60, 100, 100])

# 3. Definir las reglas difusas
reglas = [
    ctrl.Rule(velocidad['baja'] & rpm['bajas'], marcha['baja']),
    ctrl.Rule(velocidad['baja'] & rpm['medias'], marcha['media']),
    ctrl.Rule(velocidad['baja'] & rpm['altas'], marcha['media']),
    ctrl.Rule(velocidad['media'] & rpm['bajas'], marcha['media']),
    ctrl.Rule(velocidad['media'] & rpm['medias'], marcha['media']),
    ctrl.Rule(velocidad['media'] & rpm['altas'], marcha['alta']),
    ctrl.Rule(velocidad['alta'] & rpm['bajas'], marcha['media']),
    ctrl.Rule(velocidad['alta'] & rpm['medias'], marcha['alta']),
    ctrl.Rule(velocidad['alta'] & rpm['altas'], marcha['alta']),
    ctrl.Rule(pendiente['empinado'], marcha['baja']),
    ctrl.Rule(pendiente['moderado'] & carga['pesada'], marcha['baja']),
    ctrl.Rule(pendiente['moderado'] & carga['media'], marcha['baja']),
]

# 4. Crear el sistema de control difuso
sistema_control = ctrl.ControlSystem(reglas)
simulacion = ctrl.ControlSystemSimulation(sistema_control)

# 5. Función para actualizar el valor de la marcha y mostrarlo
def actualizar_simulacion(vel, rpm_value, pendiente_value, carga_value):
    simulacion.input['velocidad'] = vel
    simulacion.input['rpm'] = rpm_value
    simulacion.input['pendiente'] = pendiente_value
    simulacion.input['carga'] = carga_value
    
    simulacion.compute()

    marcha_crisp = simulacion.output['marcha']

    # Determinar la marcha a partir del valor crisp
    if marcha_crisp < 17:
        marcha_value = 1
    elif 17 <= marcha_crisp < 34:
        marcha_value = 2
    elif 34 <= marcha_crisp < 49:
        marcha_value = 3
    elif 49 <= marcha_crisp < 65:
        marcha_value = 4
    elif 65 <= marcha_crisp < 82:
        marcha_value = 5
    else:
        marcha_value = 6

    marcha_label.value = f"Marcha seleccionada: {marcha_value} (Crisp: {marcha_crisp:.2f})"

# 6. Crear los widgets
vel_slider = widgets.IntSlider(value=0, min=0, max=100, step=1, description='Velocidad:')
rpm_slider = widgets.IntSlider(value=1500, min=0, max=7000, step=1, description='RPM:')
pendiente_slider = widgets.IntSlider(value=0, min=0, max=100, step=1, description='Pendiente:')
carga_slider = widgets.IntSlider(value=0, min=0, max=100, step=1, description='Carga:')

marcha_label = widgets.Label(value="Marcha seleccionada: 1")

# 7. Enlazar los widgets a la función de actualización
widgets.interactive(actualizar_simulacion, 
                    vel=vel_slider, 
                    rpm_value=rpm_slider, 
                    pendiente_value=pendiente_slider, 
                    carga_value=carga_slider)

# 8. Mostrar los widgets
display(vel_slider, rpm_slider, pendiente_slider, carga_slider, marcha_label)


IntSlider(value=0, description='Velocidad:')

IntSlider(value=1500, description='RPM:', max=7000)

IntSlider(value=0, description='Pendiente:')

IntSlider(value=0, description='Carga:')

Label(value='Marcha seleccionada: 1 (Crisp: 15.56)')